In [1]:
import numpy as np
import inspyred as ins
from pylab import *
import sys
from inspyred import ec, benchmarks

In [ ]:
#from job import Job
#from activity import Activity
#from operation import Operation
#from machine import Machine
import re
import os

def parse(path):
    with open(os.path.join(os.getcwd(), path, "r")) as data:
        total_jobs, total_machines, max_operations = re.findall("\S+", data.readline())
        number_total_jobs, number_total_machines, number_max_operations = int(total_jobs), int(total_machines), int(float(
			max_operations))
		
        jobs_list = []
        #current job id
        id_job = 1

        for key, line in enumerate(data):
			
            if key >= number_total_jobs:
                break
			
            # Split data with multiple spaces as separator
            parsed_line = re.findall('\S+', line)
			# Current job
            job = Job(id_job)
			# Current activity's id
            id_activity = 1
			# Current item of the parsed line
            i = 1

            while i < len(parsed_line):
				# Total number of operations for the activity
                number_operations = int(parsed_line[i])
				# Current activity
                activity = Activity(job, id_activity)
                for id_operation in range(1, number_operations + 1):
                    activity.add_operation(Operation(id_operation, int(parsed_line[i + 2 * id_operation - 1]),
                                int(parsed_line[i + 2 * id_operation])))

                job.add_activity(activity)
                i += 1 + 2 * number_operations
                id_activity += 1

                jobs_list.append(job)
                id_job += 1

	# Machines
    machines_list = []
    for id_machine in range(1, number_total_machines + 1):
        machines_list.append(Machine(id_machine, number_max_operations))

    return jobs_list, machines_list, number_max_operations

In [15]:
path = r"C:\Users\saadr\Downloads\TextData.zip\Monaldo\Fjsp\Job_Data\Barnes\Text\mt01c1"

In [18]:
parse(path)

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\saadr\\Downloads\\TextData.zip\\Monaldo\\Fjsp\\Job_Data\\Barnes\\Text\\mt01c1\\r'

In [5]:
test= [6   6   1   
6   1   3   1   1   1   3   1   2   6   1   4   7   1   6   3   1   5   6   
6   1   2   8   1   3   5   1   5   10  1   6   10  1   1   10  1   4   4   
6   1   3   5   1   4   4   1   6   8   1   1   9   1   2   1   1   5   7   
6   1   2   5   1   1   5   1   3   5   1   4   3   1   5   8   1   6   9   
6   1   3   9   1   2   3   1   5   5   1   6   4   1   1   3   1   4   1   
6   1   2   3   1   4   3   1   6   9   1   1   10  1   5   4   1   3   1]

SyntaxError: invalid syntax. Perhaps you forgot a comma? (1993158878.py, line 1)

In [ ]:
m,n = test.shape

In [ ]:
jobs

## Class Jobs

In [19]:
class Job:
	def __init__(self, id_job):
		self.__id_job = id_job
		self.__activities_to_be_done = []
		self.__activities_done = []

	# Display the job nicer
	def __str__(self):
		output = ""

		for activity in self.__activities_to_be_done:
			output += str(activity) + "\n"

		for activity in self.__activities_done:
			output += str(activity) + "\n"

		return output

	# Return the job's id
	@property
	def id_job(self):
		return self.__id_job

	# Add an activity to the job
	def add_activity(self, activity):
		self.__activities_to_be_done.append(activity)

	# Return if the job is done
	@property
	def is_done(self):
		return len(self.activities_to_be_done) == 0

	# Return the activities yet to be done
	@property
	def activities_to_be_done(self):
		return self.__activities_to_be_done

	# Return the activities already done
	@property
	def activities_done(self):
		return self.__activities_done

	# Method an activity calls to signal it's done
	def activity_is_done(self, activity):
		if not activity.is_done:
			raise EnvironmentError("This activity is not done")
		self.__activities_to_be_done = list(
			filter(lambda element: element.id_activity != activity.id_activity, self.__activities_to_be_done))
		self.__activities_done.append(activity)

	# Return the current activity that need to be processe
	@property
	def current_activity(self):
		if len(self.activities_to_be_done) == 0:
			raise EnvironmentError("All activities are already done")
		return self.__activities_to_be_done[0]

	@property
	def remaining_shop_time(self):
		return sum(map(lambda activity: activity.shop_time, self.activities_to_be_done))

	@property
	def total_shop_time(self):
		return sum(map(lambda activity: activity.shop_time, self.activities_to_be_done + self.activities_done))

	def check_if_previous_activity_is_done(self, activity_id):
		if activity_id == 1:
			return True
		for activity in self.__activities_done:
			if activity.id_activity == activity_id - 1:
				return True
		return False

	def get_activity(self, id_activity):
		for activity in self.__activities_to_be_done:
			if activity.id_activity == id_activity:
				return activity


In [20]:
class Machine:
	def __init__(self, id_machine, max_operations):
		self.__id_machine = id_machine
		self.__is_working = False
		self.__operations_done = []
		self.__processed_operations = []
		self.__max_operations = max_operations
		self.__current_time = 0
		self.__available_places = [i for i in range(max_operations)]

	# Return the machine's id
	@property
	def id_machine(self):
		return self.__id_machine

	# Return the operations done by the machine
	@property
	def operations_done(self):
		return self.__operations_done

	# Return if the machine is working at max capacity
	def is_working_at_max_capacity(self):
		return len(self.__processed_operations) == self.__max_operations

	# Add an operation to the treatment list of the machine
	def add_operation(self, activity, operation):
		if self.is_working_at_max_capacity():
			raise EnvironmentError("Machine already working at max capacity")
		if operation.id_machine != self.__id_machine:
			raise EnvironmentError("Operation assigned to the wrong machine")

		operation.time = self.__current_time
		operation.is_pending = True
		operation.place_of_arrival = self.__available_places.pop(0)

		self.__processed_operations.append((activity, operation))

	# Method to simulate a work process during one unit of time
	def work(self):
		self.__current_time += 1
		for activity, operation in self.__processed_operations:
			if operation.time + operation.duration <= self.__current_time:
				self.__processed_operations = list(filter(lambda element: not (
						element[0].id_job == activity.id_job and element[0].id_activity == activity.id_activity and
						element[1].id_operation == operation.id_operation), self.__processed_operations))
				self.__available_places.append(operation.place_of_arrival)
				activity.terminate_operation(operation)
				self.__operations_done.append(operation)


In [21]:
class Operation:
	def __init__(self, id_operation, id_machine, duration):
		self.__id_operation = id_operation
		self.__duration = duration
		self.__id_machine = id_machine
		self.__time = None
		self.__is_pending = False
		self.__place_of_arrival = None

	# Display the operation nicer
	def __str__(self):
		output = "Operation n°" + str(self.__id_operation) + " -> Machine: " + str(
			self.__id_machine) + ", Duration: " + str(self.__duration)

		if not (self.__time is None):
			output += ", Started at time " + str(self.__time)

		return output

	# Return the operation's id
	@property
	def id_operation(self):
		return self.__id_operation

	# Return if an operation is done at time t
	def is_done(self, t):
		return not (self.__time is None) and self.__time + self.__duration <= t

	# Return if a machine is already treating the operation
	@property
	def is_pending(self):
		return self.__is_pending

	# Set the pending status
	@is_pending.setter
	def is_pending(self, value):
		self.__is_pending = value

	# Return the slot of the machine allocated for the treatment of the operation
	@property
	def place_of_arrival(self):
		return self.__place_of_arrival

	# Set the slot of the machine allocated for the treatment of the operation
	@place_of_arrival.setter
	def place_of_arrival(self, value):
		self.__place_of_arrival = value

	# Return the machine's id who has to do the operation
	@property
	def id_machine(self):
		return self.__id_machine

	# Return the operation's duration
	@property
	def duration(self):
		return self.__duration

	# Return at which time the operation started or None
	@property
	def time(self):
		return self.__time

	# Set the time at which the operation started
	@time.setter
	def time(self, value):
		if value < 0:
			raise ValueError("Time < 0 is not possible")
		self.__time = value


In [23]:
class Activity:
	def __init__(self, job, id_activity):
		self.__job = job
		self.__id_activity = id_activity
		self.__operations_to_be_done = []
		self.__operation_done = None

	# Display the activity nicer
	def __str__(self):
		output = "Job n°" + str(self.id_job) + " Activity n°" + str(self.__id_activity) + "\n"

		output += "Operations to be done\n"
		for operation in self.__operations_to_be_done:
			output += str(operation) + "\n"

		output += "Operation done\n"
		output += str(self.__operation_done) + "\n"

		return output

	# Return the job's id of the activity
	@property
	def id_job(self):
		return self.__job.id_job

	# Return the activity's id
	@property
	def id_activity(self):
		return self.__id_activity

	# Add an operation to the activity
	def add_operation(self, operation):
		self.__operations_to_be_done.append(operation)

	# Return if the activity is done
	@property
	def is_done(self):
		return not (self.__operation_done is None)

	# Return the list of all the operations yet to be done
	@property
	def next_operations(self):
		return self.__operations_to_be_done

	# Return the shortest operation available
	@property
	def shortest_operation(self):
		candidate_operation = None
		for operation in self.__operations_to_be_done:
			if candidate_operation is None or operation.duration < candidate_operation.duration:
				candidate_operation = operation
		return operation

	# Return the list of all the operations already done
	@property
	def operation_done(self):
		return self.__operation_done

	# Allow a machine to say to an activity that it finished an operation
	def terminate_operation(self, operation):
		# Remove the operation from the list of the operations yet to be done
		self.__operations_to_be_done = list(
			filter(lambda element: element.id_operation != operation.id_operation, self.__operations_to_be_done))
		# Append the operation to the list of the operations already done
		self.__operation_done = operation
		self.__job.activity_is_done(self)

	@property
	def shop_time(self):
		return self.operation_done.duration if self.is_done else max(self.__operations_to_be_done,
																	 key=lambda operation: operation.duration)

	@property
	def is_feasible(self):
		return self.__job.check_if_previous_activity_is_done(self.__id_activity)

	@property
	def is_pending(self):
		return len(list(filter(lambda element: element.is_pending, self.__operations_to_be_done))) > 0

	def get_operation(self, id_operation):
		for operation in self.__operations_to_be_done:
			if operation.id_operation == id_operation:
				return operation
